# Load Bronze Data to Silver Table - Account (Fabric & ADB)

## Overview
Load Account sample data from Bronze lakehouse files into Silver lakehouse table for both Fabric and ADB channels.

## Data Flow
- **Source (Fabric)**: MAAG_LH_Bronze/Files/samples_fabric/finance/Account_Samples_Fabric.csv
- **Source (ADB)**: MAAG_LH_Bronze/Files/samples_databricks/finance/Account_Samples_ADB.csv
- **Target**: MAAG_LH_Silver.finance.Account table (or any attached default lakehouse)
- **Process**: Read CSV, validate schema, check data quality, show value distributions, load to Delta table, verify load

---

In [5]:
# --- Fabric Channel ---
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql import functions as F

WORKSPACE_NAME = "Fabric_MAAG"
SOURCE_LAKEHOUSE_NAME = "MAAG_LH_Bronze"
FABRIC_SOURCE_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{SOURCE_LAKEHOUSE_NAME}.Lakehouse/Files/samples_fabric/finance/Account_Samples_Fabric.csv"

TARGET_SCHEMA = "finance"
TARGET_TABLE = "Account"
TARGET_FULL_PATH = f"{TARGET_SCHEMA}.{TARGET_TABLE}"

print(f"🔄 Loading Fabric Account data")
print(f"📂 Source: {FABRIC_SOURCE_PATH}")
print(f"🎯 Target: {TARGET_FULL_PATH}")

# Read CSV from Bronze lakehouse
account_df = spark.read.option("header", "true").option("inferSchema", "true").csv(FABRIC_SOURCE_PATH)

print(f"✅ Data loaded successfully")
print(f"📊 Records: {account_df.count()}")
print(f"📋 Columns: {account_df.columns}")

# Display sample data
print(f"\n📖 Sample data:")
account_df.show(10, truncate=False)

required_columns = [
    'AccountId', 'AccountNumber', 'CustomerId', 'AccountType', 'AccountStatus', 'CreatedDate', 'CreatedBy'
 ]

missing_columns = [c for c in required_columns if c not in account_df.columns]
if missing_columns:
    print(f"⚠️ Warning: Missing columns in source data: {missing_columns}")
else:
    print(f"✅ All required columns present in source data.")

for col_name in missing_columns:
    account_df = account_df.withColumn(col_name, F.lit(""))

account_df = account_df.withColumn('AccountId', col('AccountId').cast(StringType()))
account_df = account_df.withColumn('AccountNumber', col('AccountNumber').cast(StringType()))
account_df = account_df.withColumn('CustomerId', col('CustomerId').cast(StringType()))
account_df = account_df.withColumn('AccountType', col('AccountType').cast(StringType()))
account_df = account_df.withColumn('AccountStatus', col('AccountStatus').cast(StringType()))
account_df = account_df.withColumn('CreatedDate', col('CreatedDate').cast(DateType()))
account_df = account_df.withColumn('CreatedBy', col('CreatedBy').cast(StringType()))
account_df = account_df.select(required_columns)

# Data quality checks
print(f"\n📊 Data Quality Check:")
null_counts = account_df.select([F.sum(col(c).isNull().cast("int")).alias(c) for c in required_columns]).collect()[0]
for col_name in required_columns:
    null_count = null_counts[col_name]
    if null_count > 0:
        print(f"  {col_name}: {null_count} null values")
    else:
        print(f"  {col_name}: ✅ No nulls")

# Show value distributions for AccountStatus
print(f"\n🎯 AccountStatus Distribution:")
account_df.groupBy('AccountStatus').count().orderBy('AccountStatus').show()

# Ensure the target schema exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {TARGET_SCHEMA}")

# Load data to Silver table
print(f"💾 Loading data to Silver table: {TARGET_FULL_PATH}")
try:
    account_df.write \
        .format("delta") \
        .mode("append") \
        .option("overwriteSchema", "true") \
        .saveAsTable(TARGET_FULL_PATH)
    print(f"✅ Data loaded successfully to {TARGET_FULL_PATH}")
    # Verify the load
    result_count = spark.sql(f"SELECT COUNT(*) as count FROM {TARGET_FULL_PATH}").collect()[0]['count']
    print(f"📊 Records in target table: {result_count}")
    # Show sample of loaded data
    print(f"\n📖 Sample from Silver table:")
    spark.sql(f"SELECT * FROM {TARGET_FULL_PATH} ORDER BY AccountId").show(10, truncate=False)
    print(f"🎉 Account data load complete!")
except Exception as e:
    print(f"❌ Error loading data to table: {str(e)}")
    raise
# --- End Fabric Channel ---

StatementMeta(, 1a5d319f-f503-4b61-a84a-f279d3ed88fb, 7, Finished, Available, Finished)

🔄 Loading Fabric Account data
📂 Source: abfss://Fabric_MAAG@onelake.dfs.fabric.microsoft.com/MAAG_LH_Bronze.Lakehouse/Files/samples_fabric/finance/Account_Samples_Fabric.csv
🎯 Target: finance.Account
✅ Data loaded successfully
📊 Records: 513
📋 Columns: ['AccountId', 'AccountNumber', 'CustomerId', 'AccountType', 'AccountStatus', 'CreatedDate', 'ClosedDate', 'Balance', 'Currency', 'Description']

📖 Sample data:
+------------------------------------+---------------+----------+-----------+-------------+-----------+----------+--------+--------+------------------------------------+
|AccountId                           |AccountNumber  |CustomerId|AccountType|AccountStatus|CreatedDate|ClosedDate|Balance |Currency|Description                         |
+------------------------------------+---------------+----------+-----------+-------------+-----------+----------+--------+--------+------------------------------------+
|130853a8-c871-47f6-a825-2164ca688d23|ACC-Fabric-1000|CID-001   |Receivable |

In [4]:
# --- ADB Channel ---
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql import functions as F

WORKSPACE_NAME = "Fabric_MAAG"
SOURCE_LAKEHOUSE_NAME = "MAAG_LH_Bronze"
ADB_SOURCE_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{SOURCE_LAKEHOUSE_NAME}.Lakehouse/Files/samples_databricks/finance/Account_Samples_ADB.csv"

TARGET_SCHEMA = "finance"
TARGET_TABLE = "Account"
TARGET_FULL_PATH = f"{TARGET_SCHEMA}.{TARGET_TABLE}"

print(f"🔄 Loading ADB Account data")
print(f"📂 Source: {ADB_SOURCE_PATH}")
print(f"🎯 Target: {TARGET_FULL_PATH}")

# Read CSV from Bronze lakehouse
account_df = spark.read.option("header", "true").option("inferSchema", "true").csv(ADB_SOURCE_PATH)

print(f"✅ Data loaded successfully")
print(f"📊 Records: {account_df.count()}")
print(f"📋 Columns: {account_df.columns}")

# Display sample data
print(f"\n📖 Sample data:")
account_df.show(10, truncate=False)

required_columns = [
    'AccountId', 'AccountNumber', 'CustomerId', 'AccountType', 'AccountStatus', 'CreatedDate', 'CreatedBy'
 ]

missing_columns = [c for c in required_columns if c not in account_df.columns]
if missing_columns:
    print(f"⚠️ Warning: Missing columns in source data: {missing_columns}")
else:
    print(f"✅ All required columns present in source data.")

for col_name in missing_columns:
    account_df = account_df.withColumn(col_name, F.lit(""))

account_df = account_df.withColumn('AccountId', col('AccountId').cast(StringType()))
account_df = account_df.withColumn('AccountNumber', col('AccountNumber').cast(StringType()))
account_df = account_df.withColumn('CustomerId', col('CustomerId').cast(StringType()))
account_df = account_df.withColumn('AccountType', col('AccountType').cast(StringType()))
account_df = account_df.withColumn('AccountStatus', col('AccountStatus').cast(StringType()))
account_df = account_df.withColumn('CreatedDate', col('CreatedDate').cast(DateType()))
account_df = account_df.withColumn('CreatedBy', col('CreatedBy').cast(StringType()))
account_df = account_df.select(required_columns)

# Data quality checks
print(f"\n📊 Data Quality Check:")
null_counts = account_df.select([F.sum(col(c).isNull().cast("int")).alias(c) for c in required_columns]).collect()[0]
for col_name in required_columns:
    null_count = null_counts[col_name]
    if null_count > 0:
        print(f"  {col_name}: {null_count} null values")
    else:
        print(f"  {col_name}: ✅ No nulls")

# Show value distributions for AccountStatus
print(f"\n🎯 AccountStatus Distribution:")
account_df.groupBy('AccountStatus').count().orderBy('AccountStatus').show()

# Ensure the target schema exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {TARGET_SCHEMA}")

# Load data to Silver table
print(f"💾 Loading data to Silver table: {TARGET_FULL_PATH}")
try:
    account_df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(TARGET_FULL_PATH)
    print(f"✅ Data loaded successfully to {TARGET_FULL_PATH}")
    # Verify the load
    result_count = spark.sql(f"SELECT COUNT(*) as count FROM {TARGET_FULL_PATH}").collect()[0]['count']
    print(f"📊 Records in target table: {result_count}")
    # Show sample of loaded data
    print(f"\n📖 Sample from Silver table:")
    spark.sql(f"SELECT * FROM {TARGET_FULL_PATH} ORDER BY AccountId").show(10, truncate=False)
    print(f"🎉 Account data load complete!")
except Exception as e:
    print(f"❌ Error loading data to table: {str(e)}")
    raise
# --- End ADB Channel ---

StatementMeta(, 1a5d319f-f503-4b61-a84a-f279d3ed88fb, 6, Finished, Available, Finished)

🔄 Loading ADB Account data
📂 Source: abfss://Fabric_MAAG@onelake.dfs.fabric.microsoft.com/MAAG_LH_Bronze.Lakehouse/Files/samples_databricks/finance/Account_Samples_ADB.csv
🎯 Target: finance.Account
✅ Data loaded successfully
📊 Records: 513
📋 Columns: ['AccountId', 'AccountNumber', 'CustomerId', 'AccountType', 'AccountStatus', 'CreatedDate', 'ClosedDate', 'Balance', 'Currency', 'Description']

📖 Sample data:
+------------------------------------+-------------+----------+-----------+-------------+-----------+----------+------------------+--------+---------------------------------+
|AccountId                           |AccountNumber|CustomerId|AccountType|AccountStatus|CreatedDate|ClosedDate|Balance           |Currency|Description                      |
+------------------------------------+-------------+----------+-----------+-------------+-----------+----------+------------------+--------+---------------------------------+
|375977d2-a88e-4c47-abe7-9a8c2ac011bf|ACC-ADB-1000 |CID-001   |R